<a href="https://colab.research.google.com/github/Dwinurcahyo2/Data-Mining-Lanjut/blob/main/Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm

# Path ke folder CSV yang sudah difilter
INPUT_FILTERED_CSV_DIR = "/content/output/filtered_csv"

# Path untuk menyimpan file CSV fitur
OUTPUT_FEATURES_CSV = "/content/output/features/extracted_features.csv"

# Buat direktori output untuk fitur jika belum ada
os.makedirs(os.path.dirname(OUTPUT_FEATURES_CSV), exist_ok=True)

# Ambil daftar file CSV dari direktori input yang difilter
filtered_csv_files = [f for f in os.listdir(INPUT_FILTERED_CSV_DIR) if f.endswith('.csv')]

if not filtered_csv_files:
    print(f"Tidak ada file CSV yang difilter ditemukan di {INPUT_FILTERED_CSV_DIR}. Pastikan proses filtering sudah berhasil.")
else:
    print(f"Mengekstraksi fitur dari {len(filtered_csv_files)} file CSV yang difilter...")

    all_features = []

    # Iterasi melalui setiap file CSV yang difilter
    for file_name in tqdm(filtered_csv_files, desc="Mengekstraksi Fitur"):
        input_path = os.path.join(INPUT_FILTERED_CSV_DIR, file_name)

        try:
            df_filtered = pd.read_csv(input_path)

            # Tentukan label berdasarkan nama file (misal: 'arrhythmia_' atau 'noise_')
            label = 'noise' if 'noise_' in file_name else 'arrhythmia'

            # Pastikan DataFrame tidak kosong dan memiliki kolom sinyal
            if df_filtered.empty or len(df_filtered.columns) == 0:
                print(f"Peringatan: File {input_path} kosong atau tidak memiliki kolom. Melewati.")
                continue

            # Ekstraksi fitur untuk setiap kolom (sinyal) dalam file
            for col in df_filtered.columns:
                 # Pastikan kolom adalah tipe numerik sebelum ekstraksi fitur
                if pd.api.types.is_numeric_dtype(df_filtered[col]):
                    signal_data = df_filtered[col].values

                    # Ekstrak fitur menggunakan fungsi yang sudah ada
                    features = extract_features(signal_data)

                    # Tambahkan metadata (nama file, nama sinyal, label)
                    features['file_name'] = file_name
                    features['signal_name'] = col
                    features['label'] = label # Tambahkan label noise/arrhythmia

                    all_features.append(features)
                else:
                     print(f"Peringatan: Kolom '{col}' di file {input_path} bukan numerik. Melewati ekstraksi fitur untuk kolom ini.")


        except Exception as e:
            print(f"Gagal memproses file {input_path} saat ekstraksi fitur: {e}")

    if all_features:
        # Konversi list of dicts menjadi DataFrame
        features_df = pd.DataFrame(all_features)

        # Simpan DataFrame fitur ke file CSV
        features_df.to_csv(OUTPUT_FEATURES_CSV, index=False)

        print(f"\nProses ekstraksi fitur selesai. Fitur disimpan di: {OUTPUT_FEATURES_CSV}")
        display(features_df.head()) # Tampilkan beberapa baris pertama DataFrame fitur
    else:
        print("\nTidak ada fitur yang berhasil diekstraksi.")